## Embeddings

ယခင်ဥပမာတွင်၊ ကျွန်ုပ်တို့သည် `vocab_size` အရှည်ရှိသော အမြင့်မားသည့် bag-of-words ဗက်တာများကို အသုံးပြုခဲ့ပြီး၊ အနိမ့်အတိုင်းအတာရှိသော positional representation ဗက်တာများကို sparse one-hot representation အဖြစ် သွင်းပြောင်းခဲ့ပါသည်။ သို့သော်၊ ဒီ one-hot representation သည် မှတ်ဉာဏ်အသုံးပြုမှုအရ အကျိုးမရှိပါ။ ထို့အပြင်၊ စကားလုံးတစ်လုံးစီကို သီးခြားစီ သတ်မှတ်ထားသောကြောင့်၊ one-hot encoded ဗက်တာများသည် စကားလုံးများအကြား အဓိပ္ပါယ်ဆိုင်ရာ ဆက်စပ်မှုများကို မဖော်ပြနိုင်ပါ။

ဒီယူနစ်တွင်၊ ကျွန်ုပ်တို့သည် **News AG** ဒေတာစကင်ကို ဆက်လက်လေ့လာသွားမည်ဖြစ်သည်။ စတင်ရန်အတွက်၊ ဒေတာကို load ပြုလုပ်ပြီး ယခင်ယူနစ်မှ အဓိပ္ပါယ်ဖွင့်ဆိုချက်များကို ပြန်လည်ရယူကြပါစို့။


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Embedding ဆိုတာဘာလဲ?

**Embedding** ဆိုတဲ့အယူအဆက စကားလုံးတွေကို အနိမ့်အတိုင်းအတာရှိတဲ့ ဒေတာအမျိုးအစား (dense vectors) တွေသုံးပြီး စကားလုံးရဲ့ အဓိပ္ပါယ်ကို ကိုယ်စားပြုဖို့ ရည်ရွယ်ပါတယ်။ နောက်ပိုင်းမှာ အဓိပ္ပါယ်ရှိတဲ့ စကားလုံး embedding တွေကို ဘယ်လိုတည်ဆောက်မလဲဆိုတာကို ဆွေးနွေးပေမယ့် အခုအချိန်မှာတော့ စကားလုံး vector ရဲ့ dimensionality ကို လျှော့ချဖို့ နည်းလမ်းတစ်ခုအနေနဲ့ embedding ကို သက်သက်ထင်မြင်ပါ။

Embedding layer က စကားလုံးတစ်လုံးကို input အနေနဲ့ယူပြီး `embedding_size` ဆိုတဲ့ အတိုင်းအတာရှိတဲ့ output vector ကို ထုတ်ပေးပါတယ်။ တစ်နည်းအားဖြင့်၊ ဒါဟာ `Dense` layer နဲ့ ဆင်တူပါတယ်၊ ဒါပေမယ့် one-hot encoded vector ကို input အနေနဲ့မယူဘဲ စကားလုံးနံပါတ်ကို input အနေနဲ့ယူနိုင်ပါတယ်။

ကျွန်တော်တို့ network ရဲ့ ပထမဆုံး layer အနေနဲ့ embedding layer ကို အသုံးပြုခြင်းအားဖြင့် bag-of-words နည်းလမ်းကနေ **embedding bag** မော်ဒယ်ဆီကို ပြောင်းနိုင်ပါတယ်။ ဒီမှာ ကျွန်တော်တို့ စာသားထဲက စကားလုံးတစ်လုံးချင်းစီကို သက်ဆိုင်ရာ embedding ကို ပြောင်းပြီး၊ အဲဒီ embedding တွေကို `sum`, `average`, `max` စတဲ့ aggregate function တစ်ခုခုနဲ့ တွက်ချက်ပေးနိုင်ပါတယ်။

![Image showing an embedding classifier for five sequence words.](../../../../../translated_images/my/embedding-classifier-example.b77f021a7ee67eee.webp)

ကျွန်တော်တို့ classifier neural network ရဲ့ layer တွေက အောက်ပါအတိုင်းဖြစ်ပါတယ်-

* `TextVectorization` layer: ဒီ layer က string ကို input အနေနဲ့ယူပြီး token နံပါတ်တွေပါဝင်တဲ့ tensor ကို ထုတ်ပေးပါတယ်။ ကျွန်တော်တို့ `vocab_size` ဆိုတဲ့ reasonable vocabulary size ကို သတ်မှတ်ပြီး အသုံးနည်းတဲ့ စကားလုံးတွေကို မထည့်ပါဘူး။ Input shape က 1 ဖြစ်ပြီး Output shape က $n$ ဖြစ်ပါတယ်၊ အကြောင်းကတော့ $n$ tokens ရလို့ပါ၊ အဲဒီ token တစ်ခုချင်းစီမှာ 0 ကနေ `vocab_size` အတွင်းရှိ နံပါတ်တွေ ပါဝင်ပါတယ်။
* `Embedding` layer: ဒီ layer က $n$ နံပါတ်တွေကိုယူပြီး နံပါတ်တစ်ခုချင်းစီကို သတ်မှတ်ထားတဲ့ အရှည် (ဥပမာ 100) ရှိတဲ့ dense vector ကို လျှော့ချပေးပါတယ်။ ဒါကြောင့် $n$ shape ရှိတဲ့ input tensor ကို $n\times 100$ tensor အဖြစ် ပြောင်းပေးပါတယ်။
* Aggregation layer: ဒီ layer က tensor ရဲ့ ပထမ axis အတိုင်းအတာကို average တွက်ချက်ပေးပါတယ်၊ ဒါဟာ input sequence တစ်ခုလုံးရဲ့ စုစုပေါင်း numeric ကိုယ်စားပြုမှု ဖြစ်ပါတယ်။ ဒီ layer ကို `Lambda` layer အသုံးပြုပြီး average တွက်ချက်တဲ့ function ကို ထည့်သွင်းပေးပါမယ်။ Output shape က 100 ဖြစ်ပြီး input sequence တစ်ခုလုံးရဲ့ ကိုယ်စားပြုမှု ဖြစ်ပါတယ်။
* နောက်ဆုံး `Dense` linear classifier: Neural network ရဲ့ အဆုံး layer ဖြစ်ပါတယ်။


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` အတွင်းရှိ **output shape** ကော်လံမှာ ပထမဆုံး tensor အတိုင်းအတာ `None` ဟာ minibatch အရွယ်အစားကို ကိုယ်စားပြုပြီး ဒုတိယဟာ token အစဉ်အတိုင်းအတာရဲ့ အရှည်ကို ကိုယ်စားပြုပါတယ်။ Minibatch အတွင်းရှိ token အစဉ်တွေဟာ အရှည်တစ်ခုချင်းစီကွဲပြားပါတယ်။ ဒီအကြောင်းကို နောက်ပိုင်းအပိုင်းမှာ ဆွေးနွေးပါမယ်။

အခု network ကို လေ့ကျင့်ကြစို့:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **မှတ်ချက်** ကျွန်ုပ်တို့သည် ဒေတာ၏ အပိုင်းခွဲတစ်ခုအပေါ် အခြေခံ၍ ဗက်တိုရာဇာကို တည်ဆောက်နေပါသည်။ ဤအရာကို လုပ်ဆောင်ခြင်းသည် လုပ်ငန်းစဉ်ကို မြန်ဆန်စေရန် ဖြစ်ပြီး၊ ၎င်းသည် ကျွန်ုပ်တို့၏ စာသားမှ အားလုံးသော တိုကင်များသည် ဝေါဟာရတွင် ပါဝင်မရှိနိုင်သော အခြေအနေတစ်ခု ဖြစ်ပေါ်စေနိုင်ပါသည်။ ဤအခြေအနေတွင်၊ ၎င်းတိုကင်များကို မထည့်သွင်းဘဲ လွှဲချော်မည်ဖြစ်ပြီး၊ ၎င်းသည် တိကျမှုအနည်းငယ် လျော့နည်းစေနိုင်ပါသည်။ သို့သော်၊ အမှန်တကယ်တွင် စာသား၏ အပိုင်းခွဲတစ်ခုသည် ဝေါဟာရခန့်မှန်းမှုကောင်းမွန်မှုကို အကြောင်းဖန်တီးပေးနိုင်ပါသည်။


### အမျိုးမျိုးသော အရွယ်အစားရှိ အဆင့်များကို ကိုင်တွယ်ခြင်း

မီနီဘတ်ချ်များတွင် လေ့ကျင့်မှုအခြေအနေကို နားလည်ကြရအောင်။ အထက်ပါ ဥပမာတွင် input tensor ၏ dimension သည် 1 ဖြစ်ပြီး၊ 128-အရှည်ရှိသော မီနီဘတ်ချ်များကို အသုံးပြုပါသည်။ ထို့ကြောင့် tensor ၏ အမှန်တကယ်အရွယ်အစားမှာ $128 \times 1$ ဖြစ်သည်။ သို့သော်၊ mỗi စာကြောင်းတွင် token အရေအတွက်က မတူညီပါသည်။ `TextVectorization` layer ကို တစ်ခုတည်းသော input တွင် အသုံးပြုပါက၊ စာသားကို tokenize လုပ်ပုံအပေါ် မူတည်ပြီး၊ ပြန်လာသော token အရေအတွက်က မတူညီပါသည်။


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


သို့သော်၊ vectorizer ကို အစီအစဉ်များစွာတွင် အသုံးပြုသောအခါ၊ ၎င်းသည် တိရစက်ပုံသဏ္ဍာန်ရှိသော tensor တစ်ခုကို ထုတ်လုပ်ရမည်ဖြစ်ပြီး၊ အသုံးမပြုသော အစိတ်အပိုင်းများကို PAD token (ကျွန်ုပ်တို့၏အမှုတွင် သုည) ဖြင့် ဖြည့်ဆည်းသည်။


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

## Semantic embeddings: Word2Vec

ယခင်ဥပမာတွင် embedding layer သည် စကားလုံးများကို ဗက်တာအနေဖြင့် ဖော်ပြရန် သင်ယူခဲ့သည်။ သို့သော်၊ ဤဗက်တာများတွင် အဓိပ္ပာယ်ဆိုင်ရာ အနက်မပါရှိသေးပါ။ အဓိပ္ပာယ်ဆိုင်ရာနီးစပ်မှုရှိသော စကားလုံးများ (ဥပမာ - အနက်တူသောစကားလုံးများ) သည် ဗက်တာအကွာအဝေးတစ်ခုအတွင်း နီးစပ်နေသည့် ဗက်တာများအဖြစ် ဖော်ပြနိုင်မည့် representation ကို သင်ယူနိုင်ရမည်ဖြစ်သည်။

ဒါကို ပြုလုပ်ရန်၊ [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) ကဲ့သို့သော နည်းလမ်းတစ်ခုကို အသုံးပြု၍ စာသားအစုအဝေးကြီးတစ်ခုကို အသုံးပြုကာ embedding model ကို ကြိုတင်သင်ကြားရမည်ဖြစ်သည်။ Word2Vec သည် စကားလုံးများကို distributed representation အဖြစ် ဖော်ဆောင်ရန် အသုံးပြုသော အဓိက architecture နှစ်ခုအပေါ် အခြေခံထားသည် -

 - **Continuous bag-of-words** (CBoW): ဒီနည်းလမ်းမှာ စကားလုံးတစ်လုံးကို ပတ်ဝန်းကျင်အခြေအနေမှ ခန့်မှန်းရန် မော်ဒယ်ကို သင်ကြားသည်။ ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ ရှိနေစဉ်၊ မော်ဒယ်၏ ရည်မှန်းချက်မှာ $(W_{-2},W_{-1},W_1,W_2)$ မှ $W_0$ ကို ခန့်မှန်းရန်ဖြစ်သည်။
 - **Continuous skip-gram**: CBoW နှင့် ဆန့်ကျင်ဘက်ဖြစ်သည်။ မော်ဒယ်သည် လက်ရှိစကားလုံးကို ခန့်မှန်းရန် ပတ်ဝန်းကျင်စကားလုံးများကို အသုံးပြုသည်။

CBoW သည် မြန်ဆန်သော်လည်း၊ skip-gram သည် နှေးကွေးသော်လည်း မကြာခဏမတွေ့ရသော စကားလုံးများကို ပိုမိုကောင်းမွန်စွာ ဖော်ပြနိုင်သည်။

![CBoW နှင့် Skip-Gram algorithm များကို အသုံးပြု၍ စကားလုံးများကို ဗက်တာအဖြစ် ပြောင်းလဲနေသော ပုံ။](../../../../../translated_images/my/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News dataset ပေါ်တွင် ကြိုတင်သင်ကြားထားသော Word2Vec embedding ကို စမ်းသပ်ရန် **gensim** library ကို အသုံးပြုနိုင်သည်။ အောက်တွင် 'neural' နှင့် နီးစပ်သော စကားလုံးများကို ရှာဖွေထားသည်။

> **Note:** စကားလုံးဗက်တာများကို ပထမဆုံး ဖန်တီးစဉ်၊ ဒေါင်းလုဒ်လုပ်ရန် အချိန်အတော်ကြာနိုင်ပါသည်!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ကျွန်ုပ်တို့သည် စကားလုံးမှ ဗက်တာအင်ဘက်ဒ်ကို ထုတ်ယူနိုင်ပြီး အမျိုးအစားခွဲခြားမှု မော်ဒယ်ကို လေ့ကျင့်ရန် အသုံးပြုနိုင်ပါသည်။ အင်ဘက်ဒ်တွင် ၃၀၀ ခုသော အစိတ်အပိုင်းများရှိပါသည်၊ သို့သော် ရှင်းလင်းမှုအတွက် ဗက်တာ၏ ပထမ ၂၀ ခုသော အစိတ်အပိုင်းများကိုသာ ဤနေရာတွင် ပြသထားပါသည်။


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

အထက်ပါ ဥပမာသည် အတွင်းပိုင်း GenSym မျက်လှည့်တစ်ခုကို အသုံးပြုထားသော်လည်း အခြေခံ logic သည် အလွန်ရိုးရှင်းသည်။ Embeddings အကြောင်းတွင် စိတ်ဝင်စားဖွယ်ကောင်းသောအရာတစ်ခုမှာ embedding vectors ပေါ်တွင် သာမန်ဗက်တာလုပ်ဆောင်မှုများကို ပြုလုပ်နိုင်ပြီး၊ ၎င်းသည် စကားလုံး **အဓိပ္ပါယ်များ** ပေါ်တွင် လုပ်ဆောင်မှုများကို အကျိုးသက်ရောက်စေမည်ဖြစ်သည်။ အထက်ပါ ဥပမာကို ဗက်တာလုပ်ဆောင်မှုများအနေနှင့် ဖော်ပြနိုင်သည်- **KING-MAN+WOMAN** ကို ကိုယ်စားပြုသောဗက်တာကို တွက်ချက်ပြီး (သက်ဆိုင်သောစကားလုံးများ၏ဗက်တာကိုယ်စားပြုမှုများပေါ်တွင် `+` နှင့် `-` လုပ်ဆောင်မှုများကို ပြုလုပ်သည်)၊ ထို့နောက် အဆိုပါဗက်တာနှင့် အနီးဆုံးသော စကားလုံးကို အဘိဓာန်တွင် ရှာဖွေသည်:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: *man* နှင့် *woman* vector များတွင် coefficient များကို အနည်းငယ်ထည့်ရန် လိုအပ်ခဲ့သည် - ၎င်းတို့ကို ဖယ်ရှားကြည့်ပါက ဘာဖြစ်မည်ကို ကြည့်ရှုနိုင်ပါသည်။

အနီးဆုံး vector ကို ရှာဖွေရန်အတွက်၊ TensorFlow စက်ကိရိယာများကို အသုံးပြုကာ၊ ကျွန်ုပ်တို့၏ vector နှင့် vocabulary အတွင်းရှိ vector အားလုံးအကြား အကွာအဝေးများ၏ vector ကိုတွက်ချက်ပြီး၊ ထို့နောက် `argmin` ကို အသုံးပြုကာ အနည်းဆုံးသော စကားလုံး၏ index ကို ရှာဖွေပါသည်။


Word2Vec သည် စကားလုံး၏ အဓိပ္ပါယ်ကို ဖော်ပြရန် အလွန်ကောင်းမွန်သည့် နည်းလမ်းတစ်ခုဖြစ်သော်လည်း အောက်ပါအချက်များအပါအဝင် အနည်းအကျဉ်းများစွာရှိသည်-

* CBoW နှင့် skip-gram မော်ဒယ်များသည် **ခန့်မှန်းမှုအခြေခံ အထူးသက်မှတ်များ** ဖြစ်ပြီး ဒေသဆိုင်ရာအကြောင်းအရာကိုသာ အရေးပါစွာယူဆောင်သည်။ Word2Vec သည် အပြည်ပြည်ဆိုင်ရာအကြောင်းအရာကို အသုံးမပြုပါ။
* Word2Vec သည် စကားလုံး၏ **ပုံသဏ္ဌာန်** (morphology) ကို မထည့်သွင်းစဉ်းစားပါ၊ ဥပမာအားဖြင့် စကားလုံး၏ အဓိပ္ပါယ်သည် root ကဲ့သို့သော စကားလုံး၏ အစိတ်အပိုင်းများပေါ်မူတည်နိုင်သည်။

**FastText** သည် ဒုတိယကန့်သတ်ချက်ကို ကျော်လွှားရန် ကြိုးစားပြီး Word2Vec အပေါ် အခြေခံကာ စကားလုံးတစ်ခုစီနှင့် စကားလုံးအတွင်းရှိ အက္ခရာ n-grams များအတွက် ဗက်တာကိုယ်စားပြုမှုများကို သင်ယူသည်။ ထိုကိုယ်စားပြုမှုများ၏ တန်ဖိုးများကို သင်ကြားမှုအဆင့်တစ်ခုစီတွင် ဗက်တာတစ်ခုအဖြစ် ပျမ်းမျှတွက်ချက်သည်။ ဤနည်းလမ်းသည် သင်ကြားမှုမတိုင်မီ အလွန်များသောတွက်ချက်မှုများကို ထည့်သွင်းပေးသော်လည်း စကားလုံးအထူးသက်မှတ်များတွင် sub-word အချက်အလက်များကို encode ပြုလုပ်နိုင်စေသည်။

နောက်ထပ်နည်းလမ်းတစ်ခုဖြစ်သော **GloVe** သည် စကားလုံးအထူးသက်မှတ်များကို အခြားနည်းလမ်းဖြင့် ဆောင်ရွက်သည်။ ၎င်းသည် စကားလုံး-အကြောင်းအရာ matrix ကို factorization အပေါ် အခြေခံသည်။ ပထမဦးစွာ စကားလုံးများ၏ အကြောင်းအရာအမျိုးမျိုးတွင် ဖြစ်ပေါ်မှုအရေအတွက်ကို ရေတွက်ထားသော matrix တစ်ခုကို တည်ဆောက်ပြီး ထို့နောက် reconstruction loss ကို အနည်းဆုံးဖြစ်စေရန် အနိမ့် dimension များတွင် matrix ကို ကိုယ်စားပြုရန် ကြိုးစားသည်။

gensim library သည် ဤစကားလုံးအထူးသက်မှတ်များကို ပံ့ပိုးပေးပြီး အထက်ပါ မော်ဒယ် loading code ကို ပြောင်းလဲခြင်းဖြင့် ၎င်းတို့ကို စမ်းသပ်နိုင်သည်။


## Keras တွင် pretrained embeddings ကို အသုံးပြုခြင်း

အထက်ပါ ဥပမာကို ပြင်ဆင်ပြီး semantic embeddings (ဥပမာ - Word2Vec) ဖြင့် embedding layer ရှိ matrix ကို ကြိုတင်ဖြည့်စွက်နိုင်ပါတယ်။ Pretrained embedding ရဲ့ vocabulary နဲ့ text corpus ရဲ့ vocabulary တို့ဟာ မတူညီနိုင်ပါတယ်၊ ဒါကြောင့် တစ်ခုကို ရွေးချယ်ရပါမယ်။ ဒီမှာ Tokenizer vocabulary ကို အသုံးပြုခြင်းနဲ့ Word2Vec embeddings ရဲ့ vocabulary ကို အသုံးပြုခြင်းဆိုတဲ့ နှစ်မျိုးသော ရွေးချယ်မှုများကို လေ့လာပါမယ်။

### Tokenizer vocabulary ကို အသုံးပြုခြင်း

Tokenizer vocabulary ကို အသုံးပြုတဲ့အခါမှာ vocabulary ရဲ့ စကားလုံးတချို့မှာ Word2Vec embeddings ရှိနိုင်ပြီး တချို့မှာ မရှိနိုင်ပါ။ Vocabulary size ကို `vocab_size` ဟု သတ်မှတ်ပြီး၊ Word2Vec embedding vector ရဲ့ အလျားကို `embed_size` ဟု သတ်မှတ်ပါက embedding layer ကို `vocab_size`$\times$`embed_size` ပုံစံရှိတဲ့ weight matrix တစ်ခုအဖြစ် ကိုယ်စားပြုနိုင်ပါတယ်။ ဒီ matrix ကို vocabulary ကို ဖြတ်သွားပြီး ဖြည့်စွက်ပါမယ်။


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Word2Vec vocabulary တွင် မပါဝင်သော စကားလုံးများအတွက် ကျွန်ုပ်တို့သည် zeroes အဖြစ်ထားနိုင်ပါသည်၊ သို့မဟုတ် အလှည့်ကျ random vector တစ်ခုကို ဖန်တီးနိုင်ပါသည်။

ယခု ကျွန်ုပ်တို့သည် pretrained weights ဖြင့် embedding layer တစ်ခုကို သတ်မှတ်နိုင်ပါပြီ။


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **မှတ်ချက်**: `Embedding` ကို ဖန်တီးတဲ့အချိန်မှာ `trainable=False` ကို သတ်မှတ်ထားတာကို သတိပြုပါ၊ ဒါက Embedding layer ကို ပြန်လည်လေ့ကျင့်မနေကြောင်းကို ဆိုလိုပါတယ်။ ဒီလိုလုပ်ခြင်းကြောင့် တိကျမှုအနည်းငယ်လျော့နည်းနိုင်ပေမယ့် လေ့ကျင့်မှုအချိန်ကို ပိုမိုလျင်မြန်စေပါတယ်။

### Embedding စာလုံးစုကို အသုံးပြုခြင်း

ယခင်နည်းလမ်းမှာ ရှိနေတဲ့ ပြဿနာတစ်ခုက TextVectorization နဲ့ Embedding မှာ အသုံးပြုထားတဲ့ စာလုံးစုတွေက မတူညီကြတာဖြစ်ပါတယ်။ ဒီပြဿနာကို ဖြေရှင်းဖို့အတွက် အောက်ပါနည်းလမ်းများကို အသုံးပြုနိုင်ပါတယ်-
* Word2Vec မော်ဒယ်ကို ကျွန်တော်တို့ရဲ့ စာလုံးစုအပေါ်မှာ ပြန်လည်လေ့ကျင့်ပါ။
* Pretrained Word2Vec မော်ဒယ်က စာလုံးစုကို အသုံးပြုပြီး ကျွန်တော်တို့ရဲ့ ဒေတာစနစ်ကို တင်ပါ။ ဒေတာစနစ်ကို တင်တဲ့အချိန်မှာ အသုံးပြုမယ့် စာလုံးစုကို သတ်မှတ်နိုင်ပါတယ်။

နောက်ဆုံးနည်းလမ်းက ပိုမိုလွယ်ကူတဲ့နည်းလမ်းလိုပဲ ထင်ရပါတယ်၊ ဒါကြောင့် အဲဒီနည်းလမ်းကို လက်တွေ့ကျင့်ကြည့်ရအောင်။ အရင်ဆုံး Word2Vec embeddings မှာ ပါဝင်တဲ့ စာလုံးစုကို အသုံးပြုပြီး `TextVectorization` layer တစ်ခုကို ဖန်တီးပါ-


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Gensim စကားလုံး embedding မိုဃ်းစာကြည့်တိုက်တွင် သက်ဆိုင်ရာ Keras embedding အလွှာကို သင့်အတွက် အလိုအလျောက် ဖန်တီးပေးမည့် `get_keras_embeddings` ဟုခေါ်သော အဆင်ပြေသော လုပ်ဆောင်ချက်တစ်ခု ပါရှိသည်။


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


ကျွန်ုပ်တို့ ပိုမိုတိကျမှုမြင့်မားသော ရလဒ်များ မမြင်ရသည့် အကြောင်းအရင်းများထဲမှ တစ်ခုမှာ ကျွန်ုပ်တို့၏ ဒေတာစနစ်မှ စကားလုံးအချို့သည် pretrained GloVe vocabulary တွင် မပါဝင်သောကြောင့် အဓိကအားဖြင့် မသိမ်းဆည်းထားခြင်း ဖြစ်ပါသည်။ ဤအခြေအနေကို ကျော်လွှားရန် ကျွန်ုပ်တို့၏ ဒေတာစနစ်အပေါ် အခြေခံ၍ ကျွန်ုပ်တို့၏ embedding များကို ကိုယ်တိုင် training ပြုလုပ်နိုင်ပါသည်။


## အခြေအနေအရ အဓိပ္ပါယ်များ

Word2Vec ကဲ့သို့သော ရိုးရာ pretrained embedding ကိုယ်စားပြုမှုများ၏ အဓိက ကန့်သတ်ချက်တစ်ခုမှာ စကားလုံးတစ်လုံး၏ အဓိပ္ပါယ်တစ်ချို့ကို ဖမ်းဆီးနိုင်သော်လည်း အဓိပ္ပါယ်များကို ခွဲခြားနိုင်မှုမရှိခြင်း ဖြစ်သည်။ ဒီအရာက Downstream မော်ဒယ်များတွင် ပြဿနာများ ဖြစ်ပေါ်စေနိုင်သည်။

ဥပမာအားဖြင့် 'play' ဆိုသော စကားလုံးသည် အောက်ပါ စာကြောင်းနှစ်ခုတွင် အဓိပ္ပါယ်ကွဲပြားမှုရှိသည်။
- ကျွန်တော်က ဇာတ်လမ်း **play** ကို ရုပ်ရှင်ရုံမှာ သွားကြည့်ခဲ့တယ်။
- John က သူ့သူငယ်ချင်းတွေနဲ့ **play** လုပ်ချင်တယ်။

ကျွန်တော်တို့ ပြောခဲ့တဲ့ pretrained embeddings တွေက 'play' ဆိုတဲ့ စကားလုံးရဲ့ အဓိပ္ပါယ်နှစ်ခုလုံးကို တူညီတဲ့ embedding အနေနဲ့ ကိုယ်စားပြုထားတယ်။ ဒီကန့်သတ်ချက်ကို ကျော်လွှားဖို့ **ဘာသာစကားမော်ဒယ်** အပေါ် အခြေခံပြီး embeddings တွေကို တည်ဆောက်ဖို့ လိုအပ်တယ်။ ဘာသာစကားမော်ဒယ်က စာသားအကြီးစား corpus တစ်ခုမှာ လေ့ကျင့်ထားပြီး စကားလုံးတွေကို အခြေအနေအမျိုးမျိုးမှာ ဘယ်လိုပေါင်းစည်းနိုင်တယ်ဆိုတာ *သိ* တယ်။ အခြေအနေအရ အဓိပ္ပါယ်များကို ဆွေးနွေးခြင်းက ဒီသင်ခန်းစာရဲ့ အကျယ်အဝန်းအပြင်မှာ ရှိပေမယ့် ဘာသာစကားမော်ဒယ်များကို နောက်ယူနစ်မှာ ပြန်လည်ဆွေးနွေးမယ်။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို ကျေးဇူးပြု၍ သိရှိထားပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် ရှုလေ့လာသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအမှားများ သို့မဟုတ် အနားယူမှားမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
